## Importing Libraries and Dependencies

In [3]:
from telegram.ext import *
from io import BytesIO
import cv2
import numpy as np
import tensorflow as tf

## Token

In [43]:
with open('file.txt','r') as f:
    file = f.read()

TOKEN = file.split('\n')[2].split(':')[-2] + ':' + file.split('\n')[2].split(':')[-1]

'1948474802:AAHVmdDaxejlXycJISVUEDr9XtcjmlSE-3g'

## Custom Functions

In [49]:
def start(update, context):
    update.message.reply_text("Hello! Welcome to Diptam's Bot")

def help(update, context):
    update.message.reply_text("""
    The following commands are available : 

    /start  -> Welcome message
    /help  -> To list out all the commands
    /train  -> To train the neural network
    """)

## classify images functions

In [20]:
(X_train, Y_train),(x_test,y_test) = tf.keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 359s 2us/step


In [50]:
#print(X_train/250)
X_train , x_test = X_train/255, x_test/255  #To make the colors in the range of 0,1
class_names = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

In [51]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape = (32,32,3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [61]:
def train(update, context):
    update.message.reply_text("Model is being trained ... ")
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, Y_train, epochs=10, validation_data=(x_test,y_test))
    model.save('cidar_classifier.model')
    update.message.reply_text("Model trained successfully, You can now send a photo!")

def handle_message(update, context):
    update.message.reply_text("Please train the model and send a picture")

def handle_photo(update, context):
    #Loading image
    photo = context.bot.get_file(update.message.photo[-1].file_id)
    photo.download('test.jpg')
    f = BytesIO(photo.download_as_bytearray())
    file_bytes = np.asarray(bytearray(f.read()), dtype=np.uint8)
    #Loading image done

    img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    img = cv2.resize(img, (32,32), interpolation=cv2.INTER_AREA)

    prediction = model.predict(np.array([img / 255]))
    print(prediction)
    update.message.reply_text(f"In this image I see a {class_names[np.argmax(prediction)]}")
    

## Command Handling

In [62]:
updater = Updater(TOKEN, use_context=True)
disp = updater.dispatcher


disp.add_handler(CommandHandler("start", start))
disp.add_handler(CommandHandler("help", help))
disp.add_handler(CommandHandler("train", train))
disp.add_handler(MessageHandler(Filters.text, handle_message))
disp.add_handler(MessageHandler(Filters.photo, handle_photo))


updater.start_polling()
updater.idle()

[[0.10290009 0.11099565 0.08910169 0.09818514 0.10456085 0.09148056
  0.09738978 0.09573231 0.10835797 0.10129599]]
[[0.10394573 0.11599264 0.08810871 0.09575392 0.09893893 0.08599622
  0.09038806 0.09825335 0.12196556 0.10065687]]
